This file makes the calls to NASA's API and requests the Chunks with the correct dates

Useful link: https://github.com/nasa/AppEEARS-Data-Resources/blob/main/Python/tutorials/AppEEARS_API_Area.ipynb

In [7]:
import requests as r
import getpass, pprint, time, os, cgi, json
import geopandas as gpd
import numpy as np
import shapely
from shapely import Polygon, to_geojson
from dotenv import load_dotenv

api = 'https://appeears.earthdatacloud.nasa.gov/api/'  # Set the AρρEEARS API to a variable

load_dotenv()


prods = ['MOD13A1.061', 'MOD15A2H.061', 'DAYMET.004']
layers = [(prods[0], '_500m_16_days_EVI'), (prods[0], '_500m_16_days_NDVI'), (prods[1], "Lai_500m"), (prods[2], 'prcp'), (prods[2], 'tmax'), (prods[2], 'tmin')]

task_type = ['point','area']        # Type of task, area or point
proj = 'geographic'  # Set output projection 
outFormat = ['geotiff', 'netcdf4']  # Set output file format type
recurring = False                   # Specify True for a recurring date range
#yearRange = [2000,2016]            # if recurring = True, set yearRange, change start/end date to MM-DD

params = {'limit': 1, 'pretty': True} # Limit API response to the most recent entry, return as pretty json

# Interchangeable codes 
# bounds = np.load("../../data-training/ALL-yesterday.npy", allow_pickle=True)
bounds = np.load("../../data-training/ALL.npy", allow_pickle=True) 
# dates = np.load("../../data-training/ALL-yesterday-dates.npy", allow_pickle=True)
dates = np.load("../../data-training/ALL-dates.npy", allow_pickle=True)

In [9]:
curidx = np.load('../Requests/cur_idx.npy')[0]

In [ ]:
prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })

user = os.getenv("EARTHDATA_USERNAME")
password = os.getenv("EARTHDATA_PASSWORD")
token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert API URL, call login service, provide credentials & return json
token = token_response['token']                      # Save login token to a variable
head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

request_ids = []

curidxbegan = curidx

for nextidx in range(curidx, curidx+500):
    bound = bounds[nextidx]
    
    print(nextidx)

    xmin, xmax, ymin, ymax = bound[0], bound[1], bound[2], bound[3]
    # print(xmin, xmax, ymin, ymax)

    poly = Polygon([[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]])
    features = [{'type': 'Feature', 'properties': {}, 'geometry': shapely.geometry.mapping(poly)}]

    nps_gc = {"type":"FeatureCollection", "features":features}

    startDate = dates[curidx][5:] + "-" + dates[curidx][0:4]            # Start of the date range for which to extract data: MM-DD-YYYY
    endDate = startDate             # End of the date range for which to extract data: MM-DD-YYYY

    task = {
        'task_type': task_type[1],
        'task_name': "Bounds "+str(curidx),
        'params': {
            'dates': [
            {
                'startDate': startDate,
                'endDate': endDate
            }],
            'layers': prodLayer,
            'output': {
                    'format': {
                            'type': outFormat[0]}, 
                            'projection': proj},
            'geo': nps_gc,
        }
    }

    curidx += 1

    task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post json to the API task service, return response as json
    # task_response = {"task_id":"23132930190-4920-194-9"}
    print(task_response)

    if ('task_id' in task_response):
        task_id = task_response['task_id'] 
        request_ids.append(task_id)
    else:
        break
    
    np.save('../Requests/r1'+"_"+str(curidxbegan)+"-"+str(curidx), request_ids) # Item on right is noninclusive
    
    
np.save('../Requests/cur_idx', [curidx])


In [8]:
# Final Save Line after Finished Fetching Everything (Code went out of bounds)
np.save('../Requests/r'+str(i)+"_"+str(curidxbegan)+"-"+str(curidx), request_ids)

#### Same code but for missing.npy (some chunks neglected by the above code)

In [4]:
prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })

missing = np.load("../Requests/missing.npy")
curidxmissing = 0

user = os.getenv("EARTHDATA_USERNAME")
password = os.getenv("EARTHDATA_PASSWORD")
token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert API URL, call login service, provide credentials & return json
token = token_response['token']                      # Save login token to a variable
head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

request_ids = []

curidxbegan = curidxmissing

for getnextidx in range(curidxmissing, missing.shape[0]):
    nextidx = missing[getnextidx]
    bound = bounds[nextidx]
    
    print(nextidx)

    xmin, xmax, ymin, ymax = bound[0], bound[1], bound[2], bound[3]
    # print(xmin, xmax, ymin, ymax)

    poly = Polygon([[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]])
    features = [{'type': 'Feature', 'properties': {}, 'geometry': shapely.geometry.mapping(poly)}]

    nps_gc = {"type":"FeatureCollection", "features":features}

    startDate = dates[nextidx][5:] + "-" + dates[nextidx][0:4]            # Start of the date range for which to extract data: MM-DD-YYYY
    endDate = startDate             # End of the date range for which to extract data: MM-DD-YYYY

    task = {
        'task_type': task_type[1],
        'task_name': "Bounds "+str(nextidx),
        'params': {
            'dates': [
            {
                'startDate': startDate,
                'endDate': endDate
            }],
            'layers': prodLayer,
            'output': {
                    'format': {
                            'type': outFormat[0]}, 
                            'projection': proj},
            'geo': nps_gc,
        }
    }

    # curidx += 1

    task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post json to the API task service, return response as json
    # task_response = {"task_id":"23132930190-4920-194-9"}
    print(task_response)

    if ('task_id' in task_response):
        task_id = task_response['task_id'] 
        request_ids.append(task_id)
    else:
        break
    
    np.save('../Requests/r1'+"_MISSING_A", request_ids) # Item on right is noninclusive
    
    
np.save('../Requests/cur_idx', [curidx])


25467
{'task_id': '767af532-ea27-4c04-9e5d-a4680fa040b6', 'status': 'pending'}
25942
{'task_id': '6083dffa-4482-4704-9d93-2659bafa65a0', 'status': 'pending'}
